### Quasi-phase matching temperature finder
        Sean Keenan, 5th Year MPhys Heriot-Watt University, Edinburgh
        Quantum memories group - QPL Heriot-Watt

In [ ]:
import qpm_funcs as qpm
from numpy import round, linspace, array


In [ ]:
# specify units to work in
input = 'wave'
# reverse 'engineer' poling period using known QPM temperature
find_P = True
T_find = 52.825          # Temperature of optimum conversion
T_0 = 20                # Room temp of sample (where polling period is usually quoted)
alpha = 15.4E-6         # co-efficient of expansion for material
# input frequencies (GHz) or wavelengths (um) to use
units = [191654900, 303060900] # [1.55, 0.995] 
# Sellmeier co-efficients 
Alpha = [[5.756, 0.0983, 0.2020, 189.32, 12.52, 1.32E-2], [5.653, 0.1185, 0.2091, 89.61, 10.85, 1.97E-2]] 
Beta =  [[2.860E-6, 4.700E-8, 6.113E-8, 1.516E-4], [7.941E-7, 3.134E-8, -4.641E-9, -2.188E-6]]
# Temperature array
start = 0
stop = 100
step = 0.001
num = int(round((stop-start)/step))
temperatures = linspace(start=start, stop=stop, num=num, endpoint=True)

# create lists of wavelengths and frequencies
wavelengths, frequencies = qpm.create_lists(units, unit_type=input)
# print out visible frequency
print(f"Visible at (THz), {freqs[2]*1e-6 :.2f}")
# calculate the refractive index for ordinary (o) and extraordinary axis (e) at each wavelength and temperature
ne = [qpm.calculate_n(wavelength, alphas=Alpha[0], betas=Beta[0], T=temperatures) for wavelength in wavelengths]
no = [qpm.calculate_n(wavelength, alphas=Alpha[1], betas=Beta[1], T=temperatures) for wavelength in wavelengths]
# calculate the wavevectors for each wavelength and axis
ke = [qpm.calculate_k(wavelength, n) for wavelength, n in zip(wavelengths, ne)]
ko = [qpm.calculate_k(wavelength, n) for wavelength, n in zip(wavelengths, no)]
# find the polling period of your crystal given a known optimum temperature
Pe_0 = qpm.find_P0(T_find, temperatures, ke, T_0, alpha)
Po_0 = qpm.find_P0(T_find, temperatures, ko, T_0, alpha)
# calculate polling period over temperature range
Pe = array([qpm.calculate_P(Pe_0, temperature, alpha) for temperature in temperatures])
Po = array([qpm.calculate_P(Po_0, temperature, alpha) for temperature in temperatures])
# calculate the wavevector mismatch
delta_ke = qpm.calculate_dk(Pe, ke)
delta_ko = qpm.calculate_dk(Po_0, ko)
# plot and extract the dk = 0 temperature
qpm.plot_mismatch(temperatures, delta_ke, title='extraordinary dk')
qpm.plot_mismatch(temperatures, delta_ko, title='ordinary dk')
mp.show()

In [ ]:
import tkinter as tk
from PIL import ImageTk, Image

# create a window and title and greeting
root = tk.Tk()
root.title('QPM Solver')
root.geometry('{}x{}'.format(500, 300))

frame = tk.Frame(root)
frame.pack(fill=tk.BOTH, expand=True)

freq1_var = tk.StringVar(master=root, value="193.41935")
freq2_var = tk.StringVar(master=root, value="301.30653")
freq3_var = tk.StringVar(master=root, value='494.72588')

freq_conversion = True

def switch():
    global freq_conversion

    if freq_conversion:
        convert_button.config(image = frequency)
        freq_conversion = False
    else:
        convert_button.config(image = wavelength)
        freq_conversion = True

def submit():

    if v.get() == 0:
        freq2_var.set(freq2_var.get())
        freq3_var.set(freq3_var.get())
        freq1_var.set(float(freq3_var.get()) - float(freq3_var.get()))
        print(0)
    elif v.get() == 1:
        freq1_var.set(freq1_var.get())
        freq3_var.set(freq3_var.get())
        freq2_var.set(float(freq3_var.get()) - float(freq1_var.get()))
        print(1)
    elif v.get() == 2:
        freq1_var.set(freq1_var.get())
        freq2_var.set(freq2_var.get())
        freq3_var.set(float(freq1_var.get()) + float(freq2_var.get()))
        print(2)
    
    freq = [float(freq1_var.get()), float(freq2_var.get()), float(freq3_var.get())]
    print(freq)

v = tk.IntVar()
v.set(2)

t = tk.IntVar()

R_calc_1550 = tk.Radiobutton(master=root, text="1550 Laser", variable=v, value=0)
R_calc_995 = tk.Radiobutton(master=root, text="995 Laser", variable=v, value=1)
R_calc_606 = tk.Radiobutton(master=root, text="606 Laser", variable=v, value=2)

freq1_entry = tk.Entry(master=root, textvariable=freq1_var, background='white', foreground='red', width = 20)
freq2_entry = tk.Entry(textvariable=freq2_var, background='white', foreground='red', width = 20)
freq3_entry = tk.Entry(textvariable=freq3_var, background='white', foreground='red', width = 20)

analyse_button = tk.Button(text="Run Analysis", width=15, height=2, background='white', foreground="black", command=submit)

frequency = tk.PhotoImage(file='frequency.png')
wavelength = tk.PhotoImage(file='wavelength.png')

convert_button = tk.Button(master=root, image=frequency, border=0, command=switch)

convert_button.pack()
analyse_button.pack()
freq1_entry.pack()
freq2_entry.pack()
freq3_entry.pack()
R_calc_1550.pack()
R_calc_995.pack()
R_calc_606.pack()

root.mainloop()